In [5]:
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
import time
#coding=utf-8
import sys
jieba.set_dictionary('E:/text_mining/dict.txt')             #切換至中文繁體字庫
jieba.load_userdict("E:/text_mining/dict_keyw_new.txt")     #加入自建詞庫
jieba.load_userdict("E:/text_mining/dict_cbdic.txt")


client = MongoClient('10.120.28.20',27017)
database = client['test']
collection =database['news']

def tfIdf(content):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(content)  
    weight = X.toarray()
    features = vectorizer.get_feature_names()
    print "特徵值數量:",len(features)
    return weight, features


def getTopWeight(weight, features, resp_count, top_n):
    top_features = []
    for n in range(0,resp_count):  #迴圈參考上面的總文章數
        indices = np.argsort(weight[n])[::-1]  
        # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
        
        # 看TOP多少的詞
        #top_n = 30
        top_features.append([features[i] for i in indices[:top_n]]) #這邊的寫法會讓關鍵字中間會有空白
    return top_features


#def getTopTfidf(news_id,n):
#    indices = np.argsort(weight[news_id])[::-1]  
#    features = vectorizer.get_feature_names()     
#    top_n = n
#    top_features = [features[i] for i in indices[:top_n]]
#    return top_features,indices, weight
#
#
#def getTopWeight(news_id,n):
#    top_features,indices, weight = getTopTfidf(news_id,n)
#    #a=0
#    for i in top_features:
#        #print i,weight[news_id][indices[a]]  # weight[2000] 裡面請填入跟上面一樣的文章編號
#        #a=a+1
#        return top_features
        
def apriori(D, minSup):

    '''频繁項目集用keys表示，
    key表示項目集中的某一項，
    cutKeys表示經過修剪的某k項目集。
    C表示某k項目集的每一項在D中的支持個數
    '''

    C1 = {}
    for T in D:#跑最外面的list
        for I in T:#跑每個大list裡面的小list
            if I in C1:#如果裡面的內容有在字典裡面，有的話+1，沒有的話，加入字典初始為1
                C1[I] += 1
            else:
                C1[I] = 1

    _keys1 = C1.keys()#取出c1.keys()成為另一個暫時變數(變數名前加上_,為暫時變數)

    keys1 = []#把每個keys把他變成獨立的list然後再塞入一個list 成[[A],[B],[C].....]
    for i in _keys1:
        keys1.append([i])

    n = len(D)
    cutKeys1 = []
    for k in keys1[:]:#跑keys1裡的項目，如果項目數量大於最小支持數的話，則加入cutkeys1
        if C1[k[0]]*1.0/n >= minSup:
            cutKeys1.append(k)
    cutKeys1.sort()#排序

    keys = cutKeys1#目前項目集裡面的項目(keys)
    all_keys = []
    while keys != []:
        C = getC(D, keys)#計算目前所有個key所在T的數量
        cutKeys = getCutKeys(keys, C, minSup, D)#刪掉沒有大於最小支持數的項目keys
        for key in cutKeys:
            all_keys.append(key)
        keys = aproiri_gen(cutKeys)
    return all_keys

def getC(D, keys):
    '''對目前尚有的keys進行計數'''
    C = []
    for key in keys:
        c = 0
        for T in D:
            have = True
            for k in key:
                if k not in T:
                    have = False
            if have:
                c += 1
        C.append(c)
    return C

def getCutKeys(keys, C, minSup, D):
    '''判斷這個項目有沒有大於最小支持數'''
    for key in keys[:]:
        num = 0
        for T in D:
            if keyInT(key, T):
                num += 1
        if num * 1.0 / len(D) < minSup:
            keys.remove(key)

    return keys

def keyInT(key, T):
    '''判斷項目keys有沒有在項目集T裡面'''
    for k in key:
        if k not in T:
            return False
    return True

def aproiri_gen(keys1):
    '''連起來'''
    keys2 = []
    for k1 in keys1:
        for k2 in keys1:
            if k1 != k2:
                key = []
                for k in k1:
                    if k not in key:
                        key.append(k)
                for k in k2:
                    if k not in key:
                        key.append(k)
                key.sort()
                if key not in keys2:
                    keys2.append(key)
    return keys2

Building prefix dict from E:\text_mining\dict.txt ...
DEBUG:jieba:Building prefix dict from E:\text_mining\dict.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u144803eaa96482ea338b54ad8c7f9634.cache
DEBUG:jieba:Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u144803eaa96482ea338b54ad8c7f9634.cache
Loading model cost 0.559 seconds.
DEBUG:jieba:Loading model cost 0.559 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [12]:
time_start = time.clock()
content =[]

#拿資料
#2013-01-0X; 2013-01-1x;...可行
#201301 % 23
#201406 % 16
#20160101~05 % 17
#20150421 % 28

tag1 = "陳偉殷"
tag2 = "先發"

querry_resp = collection.find({"$and":[
            {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
        ]})
print "查詢結果的第一筆資料標題:",querry_resp[0]["title"]

for post in querry_resp: 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    #title.append(post['title'])
    
#總文章數量
count = querry_resp.count()
print "查詢結果資料總筆數:", count
time_step1 = time.clock()
print "quuerry+jieba的時間", time_step1-time_start, "秒"

weight, features = tfIdf(content)
time_step2 = time.clock()
print "計算tf-idf的時間", time_step2-time_step1, "秒"

#resp_num #querr回來的第幾篇
top_num = 20   #取tf-idf前幾高的
tf_idf_resp = getTopWeight(weight, features, count, top_num)
time_step3 = time.clock()
print "排序與取值的時間", time_step3-time_step2, "秒"

D = apriori(tf_idf_resp, 0.04) #購物籃分析
time_step4 = time.clock()
print "購物籃分析的時間", time_step4-time_step3, "秒"
print ""

print "傳回", len(D), "組關聯分析結果:"
print "與查詢詞", tag1, tag2, "常出現的詞:"
for ele in D:
    if len(ele)>2:
        print (",".join(ele))
    else:
        print ele[0]


查詢結果的第一筆資料標題: 台客真英雄
查詢結果資料總筆數: 4004
quuerry+jieba的時間 22.7715528529 秒
特徵值數量: 40104
計算tf-idf的時間 0.902512864586 秒
排序與取值的時間 2.07474318802 秒
購物籃分析的時間 0.454519338733 秒

傳回 11 組關聯分析結果:
與查詢詞 陳偉殷 先發 常出現的詞:
光芒
合約
影像
擊出
殷仔
洋基
紅襪
萬美元
金鶯
陳偉殷
馬林魚
